In [1]:
import pandas as pd
import numpy as np 
import random
from sklearn.decomposition import PCA
from tqdm import tqdm
import plotly.express as px 
import plotly.io as pio 
pio.renderers.default = "browser"
import ast

In [12]:
[1/5]*6

[0.2, 0.2, 0.2, 0.2, 0.2, 0.2]

# Rolling Periods

In [13]:
#Finding the initial portoflio from desired investemnt universe 

def calculate_pdi(num_assets, num_crytpos, etf_tickers,crypto_tickers, weekly_returns): 
        
        def meanRetAn(data):             
            Result = 1
            
            for i in data:
                Result *= (1+i)
                
            Result = Result**(1/float(len(data)/52))-1
            
            return(Result)

        pca = PCA()
        PDI_dict = {}
        samples = []
        for number in [num_assets]:
            for i in range(1,20000):
                for ii in [num_crytpos]:
                    t = random.sample(etf_tickers,number-ii)
                    crytpo = random.sample(crypto_tickers,ii)
                    t.extend(crytpo)
                    samples.append(t)
        
        seen = set()
        samples_mini = [x for x in samples if frozenset(x) not in seen and not seen.add(frozenset(x))]

        
        for i,y in tqdm(zip(samples_mini,range(1,len(samples_mini)+1))):
            #prog = int(y/len(samples_mini)*100)
            #progress_bar.progress(prog)
            #status_text.text("{}% Complete".format(prog))
            n_assets = len(i)
            portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
            port_weekly_return = weekly_returns[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            ann_ret = meanRetAn(list(port_weekly_return))
            an_cov = weekly_returns[i].cov()
            port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(52)
            corr_matrix = np.array(weekly_returns[i].corr())
            principalComponents = pca.fit(corr_matrix)
            PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
            
            PDI_dict[y] = {}
            PDI_dict[y]["PDI_INDEX"] = PDI
            PDI_dict[y]["# of Assets"] = len(i)
            PDI_dict[y]["Assets"] = i
            PDI_dict[y]["Sharpe Ratio"] = ann_ret/port_std
            PDI_dict[y]["Annual Return"] = ann_ret
            PDI_dict[y]["Annual STD"] = port_std
        

            


        PDI_DF = pd.DataFrame(PDI_dict).T
        PDI_DF["Assets"] = PDI_DF["Assets"].astype(str)
        PDI_DF["# of Assets"] = PDI_DF["# of Assets"].astype(str)
        PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
        PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
        PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
        PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)

        return PDI_DF



############################################################## Trading Strategy #################################################################################
# Trading algorithm that uses the portfolio chosen, and allocated weights accordingly
def calculate_pdi_weights( returns,return_mean_range): 

    n = len(returns.columns)
    eq = [(100/n)/100]*n
    w = []
    w.append(eq)
    for i in range(1,20000):
        weights = [random.random() for _ in range(n)]
        sum_weights = sum(weights)
        weights = [1*w/sum_weights for w in weights]
        w.append(list(np.round(weights,2)))
    weights_new = []
    for i in w:
        if i not in weights_new:
            weights_new.append(i)


    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/return_mean_range))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}

    for y,num in tqdm(zip(weights_new, range(0,len(weights_new),1))):
        
        port_ret  = returns.mul(y,axis=1).sum(axis=1)

        ann_ret = meanRetAn(list(port_ret))
        an_cov = returns.cov()
        port_std = np.sqrt(np.dot(np.array(y).T, np.dot(an_cov, y)))*np.sqrt(return_mean_range)
        corr_matrix = np.array(returns.mul(y).cov())
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1

        PDI_dict[num ] = {}
        PDI_dict[num ]["PDI_INDEX"] = PDI
        PDI_dict[num ]["# of Assets"] = len(y)
        PDI_dict[num ]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[num ]["Annual Return"] = ann_ret
        PDI_dict[num ]["weights"] = y
        PDI_dict[num ]["Annual STD"] = port_std

    df = pd.DataFrame(PDI_dict).T
    df["PDI_INDEX"] = df["PDI_INDEX"].astype(float)
    df["Sharpe Ratio"] = df["Sharpe Ratio"].astype(float)
    df["Annual Return"] = df["Annual Return"].astype(float)
    df["Annual STD"] = df["Annual STD"].astype(float)

    return df



############################################################## Trading Strategy #################################################################################
# Trading algorithm that finds new portfolios each quarter
def pca_per_weights_rolling(return_data, portfolio, interval, ret_range_mean, max_pdi_train):
        data = return_data.copy() # data containing weekly returns
        tickers = list(data.columns)
        data.index = pd.to_datetime(data.index) # Conveting the index which is date to datetime
        weeks_list = data[data.index.year > 2015].index # grabbing all index dates
        data.index = data.index.to_period(interval) # converting the index to quarterly sets
        periods = data.index.unique() # taking the unique quarters to loop

        
        list_range = [] # saving rolling periods
        list_period = periods[4:] # periods of return
        for i in range(1,21): 
            list_range.append(periods[i:4+i])

        #print(periods)
        first_period = list_period[0] # the first period of the time frame
        remaining_periods = list_period[1:] # the remianing periods for returns calculations
        pdi_rolling_periods = list_range[:-1] # all periods minus the last

        ########################################  Function for pdi ########## ########## ########## ########## ########### #########  
        def pdi_period(returns, period, weights):
            pca = PCA()
            corr_matrix = np.array(returns.loc[period].mul(weights).cov())
            principalComponents = pca.fit(corr_matrix)
            return 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
        ########## ########## ########## ##########  Mean Annual Return Function ########## ########## ########## ########## ########## 
        def meanRetAn(data):             
            Result = 1
            
            for i in data:
                Result *= (1+i)
                
            Result = Result**(1/float(len(data)/ret_range_mean))-1
            
            return(Result)

        ########## ########## ########## ##########  Portfolio Return ########## ########## ########## ########## ########## ########## 
        def port_ret(returns, period, weights): # function for calculating returns
            portfolio_weights_ew = weights
            port_return = returns.loc[period].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            return  port_return

        pdi_performance_w = [] #saving performance of max pdi
        sharpe_performance_w= [] #saving performance of optimal portfolio of max portfolio
        equal_performance = [] #saving equal weight portflio performance 
        sharpe_2_performance_w =[]

        weights_pdi_performance_w = [] #weights for max pdi allocation over time
        weights_sharpe_performance_w= [] #weights for max sharpe ratio allocation over time
        weights_sharpe_2_performance_w= []
        weights_equal_performance = [] # weights for equal portfolio over time "same all periods"

        periods_weights = [] # saving periods for the weights allocation

        pdi_weights_pdi_performance_w = [] #pdi for max pdi allocation over time
        pdi_weights_sharpe_performance_w= [] #pdi for max sharpe ratio allocation over time
        pdi_weights_sharpe_2_performance_w= []
        pdi_weights_equal_performance = [] #pdi for equal portfolio over time "same all periods"




        
        assets = [] # store asstes for all periods
        assets.append(portfolio) # appending portfolio
        ############################################################ Calculate first period ####################################################################################
        #### Weighted #####
        print(first_period)
        periods_weights.append(first_period)
        first_period_df = calculate_pdi_weights( returns = data[data.index.year == 2015],return_mean_range = 52)

        ################################################## Taking the higest PDI ##########################################################################

        id = first_period_df["PDI_INDEX"].idxmax()
        port_max_pdi_weights = first_period_df["weights"][id] # getting weights for period
        port_max_pdi_weights_pdi = first_period_df["PDI_INDEX"][id] # getting weights for period
        
        weights_pdi_performance_w.append(port_max_pdi_weights) # saving weights for period
        port_max_ret_period = port_ret(data, first_period,port_max_pdi_weights)  # calculating return for periods 
        pdi_performance_w.extend(port_max_ret_period) # saving return 
        pdi_weights_pdi_performance_w.append(port_max_pdi_weights_pdi) # calculating pdi for first period

        ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
        id_sharpe = first_period_df["Sharpe Ratio"].idxmax()
        port_max_sharpe_weights = first_period_df["weights"][id_sharpe] # getting weights for period
        port_max_sharpe_weights_pdi = first_period_df["PDI_INDEX"][id_sharpe] # getting weights for period

        weights_sharpe_performance_w.append(port_max_sharpe_weights)   # saving weights for periods
        port_max_ret_period_sharpe = port_ret(data, first_period,port_max_sharpe_weights)  # calculating return for periods 
        sharpe_performance_w.extend(port_max_ret_period_sharpe) # saving return 
        pdi_weights_sharpe_performance_w.append(port_max_sharpe_weights_pdi) # calculating pdi for first period


        ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
        if len(first_period_df[first_period_df["PDI_INDEX"] > 2]) == 0:
            mini_df = first_period_df.copy()
        else:
            mini_df = first_period_df[first_period_df["PDI_INDEX"] > 2].copy()

        id_sharpe_2 = mini_df["Sharpe Ratio"].idxmax()
        port_max_sharpe_2_weights = mini_df["weights"][id_sharpe_2] # getting weights for period
        port_max_sharpe_2_weights_pdi = mini_df["PDI_INDEX"][id_sharpe_2] # getting weights for period

        weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights)   # saving weights for periods
        port_max_ret_period_sharpe_2 = port_ret(data, first_period,port_max_sharpe_2_weights)  # calculating return for periods 
        sharpe_2_performance_w.extend(port_max_ret_period_sharpe_2) # saving return 
        pdi_weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights_pdi) # calculating pdi for first period

        ################################################################## Equal Weigths Portoflio ##########################################################################

        equal_weights = first_period_df.iloc[0]["weights"] # getting weights for period
        

        weights_equal_performance.append(equal_weights)  # saving weights for periods
        port_max_ret_period_equal = port_ret(data, first_period,equal_weights) # calculating return for periods 
        equal_performance.extend(port_max_ret_period_equal) # saving return 
        pdi_weights_equal_performance.append(max_pdi_train) # calculating pdi for first period









        ######################################################## Calculation of portfolio perfomnce #############################################################################

        for init_time, next_time in zip(pdi_rolling_periods, remaining_periods):
            ############ Portfolio Creation ##############################
            print("Rolling range for calculatio: {} - period of return: {}".format(init_time, next_time))
            PDI_DF = calculate_pdi_weights(returns = data.loc[init_time].dropna(axis=1), return_mean_range = ret_range_mean)
            periods_weights.append(next_time) # saving first period
            assets.append(portfolio) # appending portfolio

            ################################################## Taking the higest PDI ##########################################################################

            id = PDI_DF["PDI_INDEX"].idxmax()
            port_max_pdi_weights = PDI_DF["weights"][id] # getting weights for period
            port_max_pdi_weights_pdi = PDI_DF["PDI_INDEX"][id] # getting weights for period
            
            weights_pdi_performance_w.append(port_max_pdi_weights) # saving weights for period
            port_max_ret_period = port_ret(data, next_time,port_max_pdi_weights)  # calculating return for periods 
            pdi_performance_w.extend(port_max_ret_period) # saving return 
            pdi_weights_pdi_performance_w.append(port_max_pdi_weights_pdi) # calculating pdi for first period

            ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
            id_sharpe = PDI_DF["Sharpe Ratio"].idxmax()
            port_max_sharpe_weights = PDI_DF["weights"][id_sharpe] # getting weights for period
            port_max_sharpe_weights_pdi = PDI_DF["PDI_INDEX"][id_sharpe] # getting weights for period

            weights_sharpe_performance_w.append(port_max_sharpe_weights)   # saving weights for periods
            port_max_ret_period_sharpe = port_ret(data, next_time,port_max_sharpe_weights)  # calculating return for periods 
            sharpe_performance_w.extend(port_max_ret_period_sharpe) # saving return 
            pdi_weights_sharpe_performance_w.append(port_max_sharpe_weights_pdi) # calculating pdi for first period


            ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
            if len(PDI_DF[PDI_DF["PDI_INDEX"] > 2]) == 0:
                mini_df = PDI_DF.copy()
            else:
                mini_df = PDI_DF[PDI_DF["PDI_INDEX"] > 2].copy()
            id_sharpe_2 = mini_df["Sharpe Ratio"].idxmax()
            port_max_sharpe_2_weights = mini_df["weights"][id_sharpe_2] # getting weights for period
            port_max_sharpe_2_weights_pdi = mini_df["PDI_INDEX"][id_sharpe_2] # getting weights for period

            weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights)   # saving weights for periods
            port_max_ret_period_sharpe_2 = port_ret(data, next_time,port_max_sharpe_2_weights)  # calculating return for periods 
            sharpe_2_performance_w.extend(port_max_ret_period_sharpe_2) # saving return 
            pdi_weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights_pdi) # calculating pdi for first period

            ################################################################## Equal Weigths Portoflio ##########################################################################

            equal_weights = PDI_DF.iloc[0]["weights"] # getting weights for period
            

            weights_equal_performance.append(equal_weights)  # saving weights for periods
            port_max_ret_period_equal = port_ret(data, next_time,equal_weights) # calculating return for periods 
            equal_performance.extend(port_max_ret_period_equal) # saving return 
            pdi_weights_equal_performance.append(pdi_period(data,next_time,equal_weights)) # calculating pdi for first period






        performance_frame = pd.DataFrame()
        performance_frame["Time"] = weeks_list
        performance_frame["Equal Weights"] = equal_performance
        performance_frame["Max PDI Weights"] = pdi_performance_w
        performance_frame["Max Sharpe Ratio Weights - PDI 2"] = sharpe_2_performance_w
        performance_frame["Max Sharpe Ratio Weights"] = sharpe_performance_w
        performance_frame["Equal Weights Cummulative"] = performance_frame["Equal Weights"].cumsum(axis=0)
        performance_frame["Max PDI Weights Cummulative"] = performance_frame["Max PDI Weights"].cumsum(axis=0) # cummulative returns max pdi
        performance_frame["Max Sharpe Ratio Weights Cummulative"] = performance_frame["Max Sharpe Ratio Weights"].cumsum(axis=0) #cummulative return sharpe ratio
        performance_frame["Max Sharpe Ratio Weights Cummulative - PDI 2"] = performance_frame["Max Sharpe Ratio Weights - PDI 2"].cumsum(axis=0) #cummulative return sharpe ratio

        weights_frame = pd.DataFrame()
        weights_frame["Period"] = periods_weights
        weights_frame["Weights Max PDI"] = weights_pdi_performance_w
        weights_frame["Weights Max Sharpe"] = weights_sharpe_performance_w
        weights_frame["Weights Max Sharpe - PDI 2"] = weights_sharpe_2_performance_w
        weights_frame["Weights Equal"] = weights_equal_performance
        weights_frame["Weights Max PDI - PDI Rolling 52"] = pdi_weights_pdi_performance_w
        weights_frame["Weights Max sharpe - PDI Rolling 52"] = pdi_weights_sharpe_performance_w
        weights_frame["Weights Max Sharpe - PDI 2 Rolling 52"] = pdi_weights_sharpe_2_performance_w
        weights_frame["Weights equal - PDI"] = pdi_weights_equal_performance
        weights_frame["Assets"] = assets







        return performance_frame, weights_frame



In [3]:
#Getting Data
our_uni = pd.read_csv("our_uni.csv", index_col="Ticker") # our defined universe


returns_crypto = pd.read_csv("crypto_weekly.csv" , index_col="Date")
tick_crypto = list(returns_crypto.columns)
returns_crypto.index = pd.to_datetime(returns_crypto.index)
returns_merge = pd.merge(returns_crypto,returns_weekly,left_index=True,right_index=True)

In [14]:
np.random.seed(42)
random.seed(42)
#Defining training data
dicti = {"MST": ['SMH', 'SPMD', 'SLYG', 'RPV', 'XSD', 'FIW', 'XTN', 'EZM', 'EWX', 'VIOG', 'XSW', 'KIE', 'PJP', 'PRN', 'XSMO', 'KCE'], "PDI_Boot":['IWO', 'IHI', 'VIOG', 'PSJ', 'XMHQ', 'BBP', 'ARKW', 'DWAS', 'FYC', 'FEMS', 'XTN', 'KRE', 'IYH', 'IAK', 'SIZE', 'IAT']}
result = {}
for uni in list(dicti.keys()):
    result[uni] = {}
    returns_crypto = pd.read_csv("crypto_weekly.csv" , index_col="Date")
    tick_crypto = list(returns_crypto.columns)
    returns_crypto.index = pd.to_datetime(returns_crypto.index)
    returns_weekly = pd.read_csv("weeklyReturns.csv", index_col="Date") # loading returns dataframe
    returns_weekly = returns_weekly[dicti[uni]]
    etf_tick = dicti[uni]
    returns_weekly.index = pd.to_datetime(returns_weekly.index) # converting returns dataframe to datetime
    returns_merge = pd.merge(returns_crypto,returns_weekly,left_index=True,right_index=True)
    train_return = returns_merge[returns_merge.index.year <= 2015] # training on data from 2015

    for num_c in [1,2]:
        result[uni][f"num_crypto_{str(num_c)}"] = {}
        for dingo in ["Sharpe Ratio"]:
            result[uni][f"num_crypto_{str(num_c)}"][dingo] = {}
            for i in [6]:
                result[uni][f"num_crypto_{str(num_c)}"][dingo][i] = {}
                print("---------------------- Calculating diversification and performnce for training period -------------------------------")
                pdi_train = calculate_pdi(num_assets = i, num_crytpos=num_c, etf_tickers = etf_tick ,crypto_tickers=tick_crypto, weekly_returns = train_return) # training on data from 2015 - getting max PDI portflio

                id_index = pdi_train[dingo].idxmax() # getting index id for max pdi
                id_index_pdi = pdi_train["PDI_INDEX"][id_index]
                assets_port = ast.literal_eval(pdi_train.loc[id_index]["Assets"]) # max pdi portfolio
                print("---------------------- Selcted Portfolio -------------------------------")
                print(assets_port)
                print("------------------------------------------------------------------------")
                ini_porti = assets_port # starting portfolio 
                test_retuns = returns_merge # defining test data 
                test_week = test_retuns[assets_port] #returns for selected portfolio
                print("-----------------------------------  Calculating Performance of selected portfolio -----------------------------------")
                performance, weights = pca_per_weights_rolling(return_data = test_week, portfolio = ini_porti, interval = "Q", ret_range_mean = 52, max_pdi_train=id_index_pdi) # running strategy for 2016 and forward
                result[uni][f"num_crypto_{str(num_c)}"][dingo][i]["performance"] = performance
                result[uni][f"num_crypto_{str(num_c)}"][dingo][i]["weights"] = weights
                print("----------------------------------- Done -----------------------------------")
                print("----------------------------------- Access dataframes - performance and weights  -----------------------------------")

15578it [00:37, 414.33it/s]
19997it [00:30, 664.33it/s]
19999it [00:31, 642.15it/s]
19998it [00:31, 637.28it/s]
19996it [00:32, 612.21it/s]
19994it [00:32, 615.74it/s]
19997it [00:30, 663.66it/s]
19998it [00:30, 664.57it/s]
19995it [00:30, 666.23it/s]
19994it [00:30, 656.64it/s]
19995it [00:28, 709.51it/s]
19998it [00:27, 735.23it/s]
19995it [00:27, 732.86it/s]
19996it [00:31, 632.04it/s]
19994it [00:30, 649.76it/s]
19997it [00:30, 657.32it/s]
19999it [00:28, 706.08it/s]
19993it [00:30, 653.07it/s]
19997it [00:30, 664.89it/s]
19996it [00:30, 666.39it/s]
19998it [00:29, 685.35it/s]
17200it [00:41, 418.20it/s]
19995it [00:34, 583.10it/s]
19996it [00:32, 624.57it/s]
19996it [00:31, 629.38it/s]
19994it [00:27, 717.03it/s]
19994it [00:28, 697.28it/s]
19997it [00:29, 688.85it/s]
19996it [00:29, 680.35it/s]
19996it [00:29, 679.34it/s]
19996it [00:29, 681.52it/s]
19994it [00:28, 701.75it/s]
19999it [00:30, 666.03it/s]
19995it [00:29, 676.23it/s]
19995it [00:29, 673.29it/s]
19994it [00:30, 649.

---------------------- Calculating diversification and performnce for training period -------------------------------
---------------------- Selcted Portfolio -------------------------------
['XSD', 'XSW', 'SPMD', 'PJP', 'KIE', 'XVG-USD']
------------------------------------------------------------------------
-----------------------------------  Calculating Performance of selected portfolio -----------------------------------
2016Q1
Rolling range for calculatio: PeriodIndex(['2015Q2', '2015Q3', '2015Q4', '2016Q1'], dtype='period[Q-DEC]', name='Date', freq='Q-DEC') - period of return: 2016Q2
Rolling range for calculatio: PeriodIndex(['2015Q3', '2015Q4', '2016Q1', '2016Q2'], dtype='period[Q-DEC]', name='Date', freq='Q-DEC') - period of return: 2016Q3
Rolling range for calculatio: PeriodIndex(['2015Q4', '2016Q1', '2016Q2', '2016Q3'], dtype='period[Q-DEC]', name='Date', freq='Q-DEC') - period of return: 2016Q4
Rolling range for calculatio: PeriodIndex(['2016Q1', '2016Q2', '2016Q3', '2016Q

In [18]:
result['MST']['num_crypto_1']['Sharpe Ratio'][6]['weights']

,Period,Weights Max PDI,Weights Max Sharpe,Weights Max Sharpe - PDI 2,Weights Equal,Weights Max PDI - PDI Rolling 52,Weights Max sharpe - PDI Rolling 52,Weights Max Sharpe - PDI 2 Rolling 52,Weights equal - PDI,Assets
0,2016Q1,"[0.41, 0.0, 0.08, 0.33, 0.15, 0.02]","[0.29, 0.34, 0.01, 0.0, 0.03, 0.33]","[0.42, 0.01, 0.01, 0.27, 0.26, 0.02]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.167290,1.000016,2.046397,1.652735,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
1,2016Q2,"[0.51, 0.01, 0.03, 0.22, 0.18, 0.04]","[0.1, 0.04, 0.18, 0.04, 0.02, 0.62]","[0.1, 0.04, 0.18, 0.04, 0.02, 0.62]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.718727,1.000000,1.000000,1.000108,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
2,2016Q3,"[0.36, 0.16, 0.02, 0.36, 0.06, 0.03]","[0.02, 0.03, 0.19, 0.0, 0.4, 0.37]","[0.02, 0.03, 0.19, 0.0, 0.4, 0.37]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.826565,1.000021,1.000021,1.000784,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
3,2016Q4,"[0.02, 0.04, 0.23, 0.29, 0.4, 0.02]","[0.55, 0.27, 0.06, 0.01, 0.04, 0.08]","[0.55, 0.27, 0.06, 0.01, 0.04, 0.08]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.820206,1.147848,1.147848,1.001503,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
4,2017Q1,"[0.02, 0.08, 0.0, 0.39, 0.48, 0.03]","[0.29, 0.02, 0.18, 0.01, 0.44, 0.06]","[0.29, 0.02, 0.18, 0.01, 0.44, 0.06]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.806202,1.078452,1.078452,1.000043,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
5,2017Q2,"[0.22, 0.03, 0.12, 0.3, 0.32, 0.02]","[0.29, 0.21, 0.0, 0.04, 0.43, 0.02]","[0.22, 0.11, 0.09, 0.23, 0.33, 0.02]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.386354,1.580410,2.095265,1.000000,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
6,2017Q3,"[0.3, 0.04, 0.35, 0.29, 0.01, 0.01]","[0.03, 0.08, 0.06, 0.12, 0.02, 0.7]","[0.26, 0.37, 0.01, 0.01, 0.34, 0.01]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.586044,1.000000,2.120347,1.000014,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
7,2017Q4,"[0.34, 0.0, 0.21, 0.33, 0.11, 0.01]","[0.05, 0.02, 0.05, 0.03, 0.13, 0.73]","[0.27, 0.34, 0.03, 0.0, 0.35, 0.01]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.561369,1.000000,2.028171,1.000000,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
8,2018Q1,"[0.22, 0.05, 0.04, 0.26, 0.42, 0.0]","[0.12, 0.02, 0.01, 0.11, 0.02, 0.72]","[0.15, 0.25, 0.14, 0.16, 0.31, 0.0]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.561122,1.000000,2.272175,1.001251,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"
9,2018Q2,"[0.13, 0.15, 0.27, 0.2, 0.25, 0.0]","[0.11, 0.11, 0.01, 0.03, 0.04, 0.7]","[0.07, 0.25, 0.2, 0.2, 0.28, 0.0]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.248679,1.000000,2.070678,1.000429,"[XSD, XSW, SPMD, PJP, KIE, XVG-USD]"


In [9]:
plot = ["Equal Weights Cummulative",	"Max PDI Weights Cummulative",	"Max Sharpe Ratio Weights Cummulative"	,"Max Sharpe Ratio Weights Cummulative - PDI 2"]
dd = result['PDI_Boot']['num_crypto_1']['Sharpe Ratio'][6]['performance']
px.line(dd, x="Time", y=plot)

In [16]:
result['PDI_Boot']['num_crypto_1']["Sharpe Ratio"][6]['weights']

,Period,Weights Max PDI,Weights Max Sharpe,Weights Max Sharpe - PDI 2,Weights Equal,Weights Max PDI - PDI Rolling 52,Weights Max sharpe - PDI Rolling 52,Weights Max Sharpe - PDI 2 Rolling 52,Weights equal - PDI,Assets
0,2016Q1,"[0.27, 0.38, 0.24, 0.05, 0.04, 0.02]","[0.16, 0.48, 0.01, 0.02, 0.03, 0.3]","[0.16, 0.48, 0.01, 0.02, 0.03, 0.3]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.952326,1.000051,1.000051,1.527616,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
1,2016Q2,"[0.24, 0.53, 0.04, 0.08, 0.06, 0.05]","[0.02, 0.0, 0.02, 0.26, 0.1, 0.59]","[0.02, 0.0, 0.02, 0.26, 0.1, 0.59]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.649184,1.000002,1.000002,1.000236,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
2,2016Q3,"[0.03, 0.4, 0.37, 0.0, 0.15, 0.05]","[0.02, 0.03, 0.06, 0.01, 0.52, 0.35]","[0.02, 0.03, 0.06, 0.01, 0.52, 0.35]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.659911,1.000067,1.000067,1.001084,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
3,2016Q4,"[0.02, 0.35, 0.28, 0.15, 0.17, 0.03]","[0.03, 0.11, 0.01, 0.34, 0.47, 0.04]","[0.03, 0.11, 0.01, 0.34, 0.47, 0.04]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.828598,1.214699,1.214699,1.018214,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
4,2017Q1,"[0.11, 0.27, 0.17, 0.04, 0.39, 0.02]","[0.62, 0.17, 0.01, 0.09, 0.02, 0.09]","[0.62, 0.17, 0.01, 0.09, 0.02, 0.09]","[0.16666666666666669, 0.16666666666666669, 0.1...",1.814295,1.137036,1.137036,1.000115,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
5,2017Q2,"[0.11, 0.31, 0.14, 0.07, 0.35, 0.02]","[0.2, 0.42, 0.07, 0.03, 0.27, 0.01]","[0.21, 0.37, 0.2, 0.08, 0.12, 0.02]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.315010,1.466247,2.071283,1.000002,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
6,2017Q3,"[0.24, 0.15, 0.18, 0.41, 0.01, 0.01]","[0.01, 0.04, 0.14, 0.1, 0.07, 0.64]","[0.28, 0.43, 0.16, 0.04, 0.08, 0.01]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.559410,1.000000,2.086917,1.000027,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
7,2017Q4,"[0.22, 0.11, 0.19, 0.38, 0.08, 0.01]","[0.11, 0.06, 0.01, 0.06, 0.02, 0.75]","[0.28, 0.37, 0.01, 0.23, 0.1, 0.01]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.574702,1.000000,2.050964,1.000000,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
8,2018Q1,"[0.18, 0.17, 0.18, 0.2, 0.27, 0.0]","[0.03, 0.03, 0.12, 0.1, 0.01, 0.71]","[0.4, 0.49, 0.0, 0.03, 0.07, 0.01]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.605518,1.000000,2.179275,1.001364,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"
9,2018Q2,"[0.21, 0.16, 0.19, 0.21, 0.22, 0.0]","[0.01, 0.03, 0.2, 0.02, 0.03, 0.72]","[0.42, 0.43, 0.03, 0.01, 0.1, 0.01]","[0.16666666666666669, 0.16666666666666669, 0.1...",2.551835,1.000000,2.240443,1.001632,"[KRE, ARKW, BBP, PSJ, IHI, XVG-USD]"


In [19]:
for i in ['MST', 'PDI_Boot']:
    for ii in ["num_crypto_1","num_crypto_2"]:
        for iii in ["Sharpe Ratio"]:
            result[i][ii][iii][6]['performance'].to_csv(f"{i}_{ii}_{iii}_performance.csv")
            result[i][ii][iii][6]['weights'].to_csv(f"{i}_{ii}_{iii}_weights.csv")



In [24]:
for i in ['MST', 'PDI_Boot']:
    print(result[i]["num_crypto_1"]["Sharpe Ratio"][6]['weights'])

    Period                                    Weights Max PDI  \
0   2016Q1  [0.16666666666666666, 0.16666666666666666, 0.1...   
1   2016Q2                [0.25, 0.25, 0.22, 0.26, 0.0, 0.03]   
2   2016Q3                 [0.0, 0.36, 0.4, 0.04, 0.14, 0.05]   
3   2016Q4               [0.37, 0.38, 0.06, 0.09, 0.06, 0.04]   
4   2017Q1               [0.16, 0.22, 0.16, 0.24, 0.21, 0.01]   
5   2017Q2                [0.05, 0.2, 0.17, 0.25, 0.31, 0.02]   
6   2017Q3               [0.13, 0.19, 0.21, 0.26, 0.19, 0.02]   
7   2017Q4               [0.01, 0.19, 0.21, 0.32, 0.26, 0.01]   
8   2018Q1               [0.14, 0.19, 0.16, 0.26, 0.25, 0.01]   
9   2018Q2               [0.19, 0.19, 0.11, 0.25, 0.26, 0.01]   
10  2018Q3               [0.23, 0.08, 0.14, 0.29, 0.24, 0.02]   
11  2018Q4                [0.2, 0.17, 0.11, 0.23, 0.28, 0.01]   
12  2019Q1                [0.02, 0.07, 0.0, 0.43, 0.43, 0.05]   
13  2019Q2               [0.08, 0.08, 0.57, 0.04, 0.11, 0.11]   
14  2019Q3               